In [15]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is 
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, roc_curve, roc_auc_score 


# keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)
from random import randint

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [16]:
X = np.load("smiley_X.npy")
y = np.load("smiley_y.npy")
print(X.shape)
print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0)
print(x_train.shape)
print(y_train.shape)

(144, 9, 9, 1)
(144,)
(108, 9, 9, 1)
(108,)


In [17]:
model = keras.models.Sequential([
  keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(9,9,1)),
  keras.layers.MaxPooling2D(2, 2),
  keras.layers.Flatten(),
  keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 7, 7, 64)          640       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 3, 3, 64)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 576)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                5770      
                                                                 
Total params: 6,410
Trainable params: 6,410
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 6ms/step - loss: 2.2888 - accuracy: 0.0741
Epoch 2/5
4/4 [==============================] - 0s 6ms/step - loss: 2.0319 - accuracy: 0.6019
Epoch 3/5
4/4 [==============================] - 0s 6ms/step - loss: 1.7818 - accuracy: 0.5833
Epoch 4/5
4/4 [==============================] - 0s 5ms/step - loss: 1.5389 - accuracy: 0.6019
Epoch 5/5
4/4 [==============================] - 0s 6ms/step - loss: 1.3069 - accuracy: 0.6759


In [19]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=0)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.7222222089767456


In [20]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 3ms/step


In [21]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 17, FP: 0, FN: 10, TP: 9
Precision: 1.0
Recall: 0.47368421052631576
f1 score: 0.6428571428571429
Sensitivity: 0.47368421052631576
Specificity: 1.0
AURC: 0.7368421052631579


In [22]:
# Architecture 2

In [23]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(9,9,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 7, 7, 32)          320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 3, 3, 32)         0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 1, 1, 64)          18496     
                                                                 
 flatten_4 (Flatten)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 256)               16640     
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                      

In [24]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 7ms/step - loss: 1.5740 - accuracy: 0.3889
Epoch 2/5
4/4 [==============================] - 0s 7ms/step - loss: 1.3663 - accuracy: 0.6667
Epoch 3/5
4/4 [==============================] - 0s 6ms/step - loss: 1.0364 - accuracy: 0.8333
Epoch 4/5
4/4 [==============================] - 0s 7ms/step - loss: 0.6947 - accuracy: 0.9167
Epoch 5/5
4/4 [==============================] - 0s 8ms/step - loss: 0.4994 - accuracy: 0.9259


In [25]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=0)
print('\nTest accuracy:', test_acc)


Test accuracy: 0.8888888955116272


In [26]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 4ms/step


In [27]:
tn, fp, fn, tp = confusion_matrix(y_test, np.argmax(y_pred, axis=1)).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_test, np.argmax(y_pred, axis=1)))
print("Recall:", recall_score(y_test, np.argmax(y_pred, axis=1)))
print("f1 score:", f1_score(y_test, np.argmax(y_pred, axis=1)))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_test, np.argmax(y_pred, axis=1)))

TN: 13, FP: 4, FN: 0, TP: 19
Precision: 0.8260869565217391
Recall: 1.0
f1 score: 0.9047619047619047
Sensitivity: 1.0
Specificity: 0.7647058823529411
AURC: 0.8823529411764706
